# Plotting orbits of two objects using RK 4 solver

## Konstantinos Doran SN:22007700 Date:21/02/24


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def force(r1,r2,m1,m2):
    """Returns the gravitational force on a mass, m1, at a point r1, due to another mass, m2, at a point r2.
    
    Inputs:
    r1      position of first mass as a numpy array
    r2      position of second mass as a numpy array
    m1      mass of first object
    m2      mass of second object    
    Returns:
    f_on_m1 force on mass 1 as a numpy array
    """
    # Find r_12 and its magnitude
    r12 = r2 - r1
    f_on_m1 = G*m1*m2/(np.sqrt((np.dot(r12,r12))**3))*r12
    return f_on_m1

In [ ]:
# Set up parameters
Ndim = 2     #2D problem for now
Nsteps = 30000
Nbodies = 2
G = 1        # Appropriate units
dt = 0.001
m1 = 0.5  # Small
m2 = 1.0     # Large
masses = np.array([m1,m2])
# Initial conditions
r1 = np.array((-2/3,0))
v1 = np.array((0,np.sqrt(2/3)))
r2 = np.array((1/3,0))
v2 = np.array((0,-np.sqrt(1/6)))
y0 = np.array((r1,r2,v1,v2))



Completely restructured my array into a singular array so that it is easier to manage positions and velocities with RK4 Solver


In [ ]:
def RK4_solver(fun,y0,dt,N):
    """Solve dy/dt = fun(y,t) using fourth-order RK method.
    Inputs:
    fun  f(y,t)
    y0   Initial condition array - positions and velocities of each object in the form (r1,v1,r2,v2,....)
    dt   Step size
    N    Number of steps
    Returns: two arrays of length N+1 (x and v or equivalent)
    """
    # Storage for solution
    #State vector consisting of positions and velocities of each body at each time
    # E.g   state vector at t0 = ((x1,y1), (x2,y2), (vx1,vy1), (vx2,vy2)) or simply (r1,r2,v1,v2)
    y = np.zeros((Nsteps,2*Nbodies,Ndim))
    # Initial condition
    y[0] = y0
    #print(y[0])
    t = 0
    for i in range(N-1):
        # RK4 formulae
        #print(y[i,0])
        k1 = dt*fun(y[i],t)
        k2 = dt*fun(y[i]+0.5*k1,t+0.5*dt)
        k3 = dt*fun(y[i]+0.5*k2,t+0.5*dt)
        #print(k3, y[i])
        k4 = dt*fun(y[i]+k3,t+dt)
        y[i+1]=y[i]+(k1+2*k2+2*k3+k4)/6
        t += dt
    return y

In [ ]:
def RHS_Orbital_motion(y,t):
    """Implements gravitational force of n bodies RHS for ODE solver
    
    Inputs:
    y_n   state vector containing r and v for each object
    t     Time (unused)
    
    Output:
    dy_n  state vector containing gradients dr and dv
    """
    statevec = np.roll(y,Ndim, axis=0)
    print(statevec)
    # for j in range(0,Nbodies-1):
    #     #Differentiate positions so velocities now replace positions in state vector
    #     forcesum = 0
    #     for i in range(0,Nbodies-1):
    #         if i!=j:
    #             forcesum += force(y[j],y[i],masses[j],masses[i])
    #             #print(forcesum)
    #     statevec[j] = y[Nbodies+j]

    f12 = force(y[0],y[1],masses[0], masses[1])      
    f21 = -f12  
    accel1 = f12/masses[0]
    accel2 = f21/masses[1]
    statevec[2] = accel1
    statevec[3] = accel2
    print(statevec)
    # statevec = (v1,v2,a1,a2)
    return statevec



In [ ]:
t_arr = np.linspace(0,Nsteps*dt,Nsteps)
orbits = RK4_solver(RHS_Orbital_motion,y0,dt,Nsteps)



In [ ]:
#plt.plot(t_arr, orbits[:,0])
plt.plot(orbits[:,0,0], orbits[:,0,1])
plt.plot(orbits[:,1,0], orbits[:,1,1])
plt.axis("scaled")
print(orbits[0,3,:])

In [ ]:
TotE = np.zeros((Nsteps))
KE = np.zeros((Nbodies,Nsteps))
PE = np.zeros((Nbodies,Nsteps))
KE[0] = 0.5*masses[0]*np.linalg.norm(orbits[:,2], axis=1)*np.linalg.norm(orbits[:,2], axis=1)
KE[1] = 0.5*masses[1]*np.linalg.norm(orbits[:,3], axis=1)*np.linalg.norm(orbits[:,3], axis=1)
r12 = np.linalg.norm(orbits[:,1]-orbits[:,0], axis=1)
r12i = 1.0/r12
#print(-G*masses[0]*masses[1]*r12i)
PE[0] = -G*masses[0]*masses[1]*r12i
PE[1] = -G*masses[0]*masses[1]*r12i
TotE = KE[0]+PE[0]+ KE[1]+PE[1]
print(TotE)
plt.plot(t_arr,TotE)

In [ ]:
L = np.zeros((Nsteps))
L1 = np.cross(orbits[:,0],orbits[:,2])*masses[0]
L2 = np.cross(orbits[:,1],orbits[:,3])*masses[1]
L = L1 +L2
print(L)
plt.plot(t_arr, L)